In [48]:
:dep ndarray = { version = "0.15.6"}
:dep plotters = { version = "0.3.5", default_features = false, features = ["evcxr", "all_series"] }

use ndarray::{Array, Zip};
use plotters::prelude::*;

In [37]:
fn f(x: f64) -> f64 {
    3.0*x.powi(2) - 4.0*x + 5.0
}

In [38]:
f(3.0)

20.0

In [40]:
let xs = Array::range(-5., 5., 0.25);

In [41]:
let ys = xs.map(|&x| f(x));

In [42]:
ys

[100.0, 91.6875, 83.75, 76.1875, 69.0, 62.1875, 55.75, 49.6875, 44.0, 38.6875, 33.75, 29.1875, 25.0, 21.1875, 17.75, 14.6875, 12.0, 9.6875, 7.75, 6.1875, 5.0, 4.1875, 3.75, 3.6875, 4.0, 4.6875, 5.75, 7.1875, 9.0, 11.1875, 13.75, 16.6875, 20.0, 23.6875, 27.75, 32.1875, 37.0, 42.1875, 47.75, 53.6875], shape=[40], strides=[1], layout=CFcf (0xf), const ndim=1

In [68]:
evcxr_figure((680, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(30)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(-5.0..5.0, 0.0..100.0)?;
    chart.configure_mesh().draw()?;
    chart.draw_series(LineSeries::new(xs.map(|&x| (x, f(x))), &BLACK))?;
    chart.configure_series_labels()
        .border_style(&BLACK).draw()?;
    root.present()?;
    Ok(())
})

0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
 
 
-5.0
 
 
 
-4.0
 
 
 
-3.0
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0

In [80]:
let h = 0.00000001;
let x = 2.0/3.0;
(f(x+h) - f(x)) / h

0.0

In [81]:
let a = 2.0;
let b = -3.0;
let c = 10.0;
let d = a*b +c;
d

4.0

In [87]:
let h = 0.0001;

// Inputs
let mut a = 2.0;
let mut b = -3.0;
let mut c = 10.0;

let d1 = a*b + c;
c += h;
let d2 = a*b + c;

println!("d1 {d1}");
println!("d2 {d2}");
// Normalized by h to get the slope
println!("slope {}", (d2 - d1)/h);

d1 4
d2 4.0001
slope 0.9999999999976694
